In [1]:
from unified_planning.io.pddl_reader import PDDLReader
from unified_planning.model import Problem
from unified_planning.shortcuts import *
from bitblast.utils import *
from pathlib import Path
import os
import numpy as np

In [2]:
domain_path = Path(os.path.abspath('')).parent / "test" / "pddl" / "simple" / "domain.pddl"
problem_path = Path(os.path.abspath('')).parent / "test" / "pddl" / "simple" / "problem.pddl"

In [3]:
reader = PDDLReader()
problem = reader.parse_problem(domain_path, problem_path)

In [4]:
numeric_variables = get_numeric_variables(problem)

action_effect_map = get_action_effect_map(problem)
constants, init_constants = get_constants(problem, get_all_eff_num(problem))

constants_abs = {np.abs(q) for q in constants.union(init_constants)}
min_bits = int(np.ceil(np.log2(max(constants_abs) + 1)))

chosen_bits = 4
assert chosen_bits >= min_bits

bitblasted_variables, bitblasted_constants = get_bitblasted_variables(numeric_variables, constants, chosen_bits)
new_initial_values = get_bitblasted_initial_state(bitblasted_variables, bitblasted_constants, problem, chosen_bits)

In [14]:
action = problem.actions[0]
assert isinstance(action, InstantaneousAction)
new_action = InstantaneousAction(action.name)
eff = effects_num(action=action).pop()
eff

# Effect conversion


x += 1

In [27]:
x = Fluent("x", RealType())
y = Fluent("y", RealType())
z = Fluent("z", RealType())

eff = Effect(condition=TRUE(), fluent=FluentExp(x), value=Int(5), kind=EffectKind.INCREASE)
eff

x += 5

In [6]:
new_problem = Problem(name='compiled')

for var, var_bits in bitblasted_variables.items():
    new_problem.add_fluents(var_bits)
    for var_bit in var_bits:
        initital_value = new_initial_values[var_bit]
        new_problem.set_initial_value(var_bit, initital_value)

for const, const_bits in bitblasted_constants.items():
    new_problem.add_fluents(const_bits)
    for const_bit in const_bits:
        initital_value = new_initial_values[const_bit]
        new_problem.set_initial_value(const_bit, initital_value)


new_problem

problem name = compiled

fluents = [
  bool x_3
  bool x_2
  bool x_1
  bool x_0
  bool q0_3
  bool q0_2
  bool q0_1
  bool q0_0
]

actions = [
]

initial fluents default = [
]

initial values = [
  x_3 := true
  x_2 := false
  x_1 := true
  x_0 := true
  q0_3 := false
  q0_2 := false
  q0_1 := false
  q0_0 := true
]

goals = [
]
